# Import Packages

In [1]:
import sys
sys.path.append('..')

import utils
import pandas as pd
import numpy as np

from mip import Model, xsum,INTEGER, CONTINUOUS, maximize, OptimizationStatus

# Import Data



In [2]:
# Import Forecasts from File System
forecasts = pd.read_csv(utils.get_project_root()+'/03_Forecasting_LightGBM/forecasts/forecasts.csv')
forecasts

,Week,Year,Site_ID,SKU,Prediction,Turnover
0,47,2020,101,1014557,30.868194,29.9
1,47,2020,101,1014824,24.796745,0.0
2,47,2020,101,1016525,56.998503,0.0
3,47,2020,107,1016525,53.501745,0.0
4,47,2020,590,1016525,55.250172,0.0
...,...,...,...,...,...,...
59869,47,2020,102,1281324700,0.767912,0.0
59870,47,2020,106,1281324700,0.767912,0.0
59871,47,2020,115,1281324700,0.767912,0.0
59872,47,2020,117,1281324700,0.767912,0.0


# Optimization (Example based on Article 1016525)

Problem:  
We would like to pass on a certain number of items from the central warehouse to the stores, but we don't have enough of them. Therefore, we need to reduce the number each store receives using an optimization.

Optimization is executed with the MIP solver (https://docs.python-mip.com/en/latest/index.html)

In [3]:
sku = 1016525
supply = 300 # stock at distribution center

In [4]:
df = forecasts.loc[forecasts.SKU == sku]
df.sort_values(by='Site_ID')

,Week,Year,Site_ID,SKU,Prediction,Turnover
2,47,2020,101,1016525,56.998503,0.0
5429,47,2020,102,1016525,52.571911,0.0
5430,47,2020,103,1016525,34.659034,0.0
5431,47,2020,106,1016525,47.174494,0.0
3,47,2020,107,1016525,53.501745,0.0
5432,47,2020,115,1016525,35.900833,0.0
5433,47,2020,116,1016525,45.830675,0.0
26946,47,2020,117,1016525,8.075953,0.0
4,47,2020,590,1016525,55.250172,0.0


In [5]:
# get overall demand
demand = df.Prediction.sum()

if supply >= demand:
    raise ValueError('supply > demand -> no optimization needed.')
    
# Get demand of each site
c_g101 = float(df.loc[df.Site_ID == 101]['Prediction'])
c_g102 = float(df.loc[df.Site_ID == 102]['Prediction'])
c_g103 = float(df.loc[df.Site_ID == 103]['Prediction'])
c_g106 = float(df.loc[df.Site_ID == 106]['Prediction'])
c_g107 = float(df.loc[df.Site_ID == 107]['Prediction'])
c_g115 = float(df.loc[df.Site_ID == 115]['Prediction'])
c_g116 = float(df.loc[df.Site_ID == 116]['Prediction'])
c_g117 = float(df.loc[df.Site_ID == 117]['Prediction'])
c_g590 = float(df.loc[df.Site_ID == 590]['Prediction'])
    
    

# Initiate the optimization-model
m = Model()


# Add Variables
x_g101 = m.add_var(var_type=INTEGER, lb=0, ub=c_g101)
x_g102 = m.add_var(var_type=INTEGER, lb=0, ub=c_g102)
x_g103 = m.add_var(var_type=INTEGER, lb=0, ub=c_g103)
x_g106 = m.add_var(var_type=INTEGER, lb=0, ub=c_g106)
x_g107 = m.add_var(var_type=INTEGER, lb=0, ub=c_g107)
x_g115 = m.add_var(var_type=INTEGER, lb=0, ub=c_g115)
x_g116 = m.add_var(var_type=INTEGER, lb=0, ub=c_g116)
x_g117 = m.add_var(var_type=INTEGER, lb=0, ub=c_g117)
x_g590 = m.add_var(var_type=INTEGER, lb=0, ub=c_g590)

p = m.add_var(var_type=CONTINUOUS, lb=0, ub=1)


# Define Constraints (dt. Nebenbedingungen)
m.add_constr( x_g101 + x_g102 + x_g103 + x_g106 + x_g107 + x_g115 + x_g116 + x_g117 + x_g590 <= supply)

m.add_constr(c_g101 * p <= x_g101)
m.add_constr(c_g102 * p <= x_g102)
m.add_constr(c_g103 * p <= x_g103)
m.add_constr(c_g106 * p <= x_g106)
m.add_constr(c_g107 * p <= x_g107)
m.add_constr(c_g115 * p <= x_g115)
m.add_constr(c_g116 * p <= x_g116)
m.add_constr(c_g117 * p <= x_g117)
m.add_constr(c_g590 * p <= x_g590)


# Define Target function
m.objective = maximize(p)


# Optimize
status = m.optimize()


# Check if optimal solution was found
if not status == OptimizationStatus.OPTIMAL:
    raise AssertionError('Optimal solution was not found.')

In [6]:
print('p: ' + str(p.x))


print('x_g101: ' + str(x_g101.x))
print('x_g102: ' + str(x_g102.x))
print('x_g103: ' + str(x_g103.x))
print('x_g106: ' + str(x_g106.x))
print('x_g107: ' + str(x_g107.x))
print('x_g115: ' + str(x_g115.x))
print('x_g116: ' + str(x_g116.x))
print('x_g117: ' + str(x_g117.x))
print('x_g590: ' + str(x_g590.x))

p: 0.7601786246804577
x_g101: 44.0
x_g102: 40.0
x_g103: 27.0
x_g106: 36.0
x_g107: 41.0
x_g115: 28.0
x_g116: 35.0
x_g117: 7.0
x_g590: 42.0


In [7]:
df.sort_values(by='Site_ID')

,Week,Year,Site_ID,SKU,Prediction,Turnover
2,47,2020,101,1016525,56.998503,0.0
5429,47,2020,102,1016525,52.571911,0.0
5430,47,2020,103,1016525,34.659034,0.0
5431,47,2020,106,1016525,47.174494,0.0
3,47,2020,107,1016525,53.501745,0.0
5432,47,2020,115,1016525,35.900833,0.0
5433,47,2020,116,1016525,45.830675,0.0
26946,47,2020,117,1016525,8.075953,0.0
4,47,2020,590,1016525,55.250172,0.0
